In [1]:
import pandas as pd

In [33]:
df = pd.read_csv('survey_results_public.csv', index_col='Respondent')
schema_df = pd.read_csv('survey_results_schema.csv', index_col='Column')

In [4]:
pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

In [5]:
# aggregation methods will only return the results of numeric columns
df.median()

CompTotal        62000.0
ConvertedComp    57287.0
WorkWeekHrs         40.0
CodeRevHrs           4.0
Age                 29.0
dtype: float64

In [6]:
df.describe()

,CompTotal,ConvertedComp,WorkWeekHrs,CodeRevHrs,Age
count,5.594500e+04,5.582300e+04,64503.000000,49790.000000,79210.000000
mean,5.519014e+11,1.271107e+05,42.127197,5.084308,30.336699
std,7.331926e+13,2.841523e+05,37.287610,5.513931,9.178390
min,0.000000e+00,0.000000e+00,1.000000,0.000000,1.000000
25%,2.000000e+04,2.577750e+04,40.000000,2.000000,24.000000
50%,6.200000e+04,5.728700e+04,40.000000,4.000000,29.000000
75%,1.200000e+05,1.000000e+05,44.750000,6.000000,35.000000
max,1.000000e+16,2.000000e+06,4850.000000,99.000000,99.000000


In [9]:
# count is the number of non NA/mising  rows
df['ConvertedComp'].count()

55823

In [10]:
len(df['ConvertedComp'])

88883

### Value Counts

In [7]:
df['Hobbyist'].value_counts()

Yes    71257
No     17626
Name: Hobbyist, dtype: int64

In [11]:
# breakdown the counts into percentage
df['Hobbyist'].value_counts(normalize=True)

Yes    0.801694
No     0.198306
Name: Hobbyist, dtype: float64

### Unique Values of a column

In [67]:
df['YearsCode'].unique()

array(['4', nan, '3', '16', '13', '6', '8', '12', '2', '5', '17', '10',
       '14', '35', '7', 'Less than 1 year', '30', '9', '26', '40', '19',
       '15', '20', '28', '25', '1', '22', '11', '33', '50', '41', '18',
       '34', '24', '23', '42', '27', '21', '36', '32', '39', '38', '31',
       '37', 'More than 50 years', '29', '44', '45', '48', '46', '43',
       '47', '49'], dtype=object)

# Grouping

In [12]:
df.groupby(['Country'])

In [13]:
country_grp = df.groupby(['Country'])

In [20]:
# return a series groupby object of the aggregated data on country and social media level
country_grp['SocialMedia'].value_counts()

Country      SocialMedia             
Afghanistan  Facebook                    15
             YouTube                      9
             I don't use social media     6
             WhatsApp                     4
             Instagram                    1
                                         ..
Zimbabwe     Facebook                     3
             YouTube                      3
             Instagram                    2
             LinkedIn                     2
             Reddit                       1
Name: SocialMedia, Length: 1220, dtype: int64

In [21]:
country_grp['SocialMedia'].value_counts().loc['China']

SocialMedia
WeChat 微信                   403
YouTube                      53
Weibo 新浪微博                   42
I don't use social media     27
Twitter                      27
Reddit                       12
LinkedIn                     11
Facebook                      8
Instagram                     7
Youku Tudou 优酷                7
WhatsApp                      3
VK ВКонта́кте                 1
Name: SocialMedia, dtype: int64

In [22]:
country_grp['ConvertedComp'].median()

Country
Afghanistan                               6222.0
Albania                                  10818.0
Algeria                                   7878.0
Andorra                                 160931.0
Angola                                    7764.0
                                          ...   
Venezuela, Bolivarian Republic of...      6384.0
Viet Nam                                 11892.0
Yemen                                    11940.0
Zambia                                    5040.0
Zimbabwe                                 19200.0
Name: ConvertedComp, Length: 179, dtype: float64

In [23]:
country_grp['ConvertedComp'].median().loc['Germany']

63016.0

In [24]:
# return multiple aggregatation functions
country_grp['ConvertedComp'].agg(['median','mean'])

,median,mean
Country,,
Afghanistan,6222.0,101953.333333
Albania,10818.0,21833.700000
Algeria,7878.0,34924.047619
Andorra,160931.0,160931.000000
Angola,7764.0,7764.000000
...,...,...
"Venezuela, Bolivarian Republic of...",6384.0,14581.627907
Viet Nam,11892.0,17233.436782
Yemen,11940.0,16909.166667


In [25]:
# sum can also work on boolean data. it counts truths as one and falses as zero

In [29]:
country_grp['LanguageWorkedWith'].apply(lambda x:x.str.contains('Python').sum())

Country
Afghanistan                              8
Albania                                 23
Algeria                                 40
Andorra                                  0
Angola                                   2
                                        ..
Venezuela, Bolivarian Republic of...    28
Viet Nam                                78
Yemen                                    3
Zambia                                   4
Zimbabwe                                14
Name: LanguageWorkedWith, Length: 179, dtype: int64

In [27]:
# country_grp is a series group by object, it can't use some series methods.
country_grp['LanguageWorkedWith'].str.contains('Python').sum()

AttributeError: Cannot access attribute 'str' of 'SeriesGroupBy' objects, try using the 'apply' method

In [30]:
country_grp['LanguageWorkedWith'].apply(lambda x:x.str.contains('Python').value_counts(normalize=True))

Country           
Afghanistan  False    0.794872
             True     0.205128
Albania      False    0.722892
             True     0.277108
Algeria      False    0.682540
                        ...   
Yemen        True     0.176471
Zambia       False    0.666667
             True     0.333333
Zimbabwe     False    0.641026
             True     0.358974
Name: LanguageWorkedWith, Length: 335, dtype: float64

### Concatenate 

In [37]:
country_respondents = df['Country'].value_counts()
country_use_python = country_grp['LanguageWorkedWith'].apply(lambda x:x.str.contains('Python').sum())

In [38]:
python_df = pd.concat([country_respondents,country_use_python], axis='columns', sort=False)
python_df

,Country,LanguageWorkedWith
United States,20949,10083
India,9061,3105
Germany,5866,2451
United Kingdom,5737,2384
Canada,3395,1558
...,...,...
North Korea,1,0
Timor-Leste,1,1
Saint Vincent and the Grenadines,1,0
Niger,1,1


In [39]:
python_df['PctKnowsPython'] = (python_df['LanguageWorkedWith']/python_df['Country']) * 100

In [40]:
python_df

,Country,LanguageWorkedWith,PctKnowsPython
United States,20949,10083,48.131176
India,9061,3105,34.267741
Germany,5866,2451,41.783157
United Kingdom,5737,2384,41.554820
Canada,3395,1558,45.891016
...,...,...,...
North Korea,1,0,0.000000
Timor-Leste,1,1,100.000000
Saint Vincent and the Grenadines,1,0,0.000000
Niger,1,1,100.000000


In [41]:
python_df.sort_values(by='PctKnowsPython', ascending=False)

,Country,LanguageWorkedWith,PctKnowsPython
Dominica,1,1,100.000000
Niger,1,1,100.000000
Timor-Leste,1,1,100.000000
Sao Tome and Principe,1,1,100.000000
Turkmenistan,7,6,85.714286
...,...,...,...
Cape Verde,3,0,0.000000
Gabon,2,0,0.000000
Liberia,2,0,0.000000
Malawi,2,0,0.000000


# Cleaning Data

In [43]:
import pandas as pd
import numpy as np

In [44]:
people = {
    'first': ['Corey', 'Jane', 'John', 'Chris', np.nan, None, 'NA'], 
    'last': ['Schafer', 'Doe', 'Doe', 'Schafer', np.nan, np.nan, 'Missing'], 
    'email': ['CoreyMSchafer@gmail.com', 'JaneDoe@email.com', 'JohnDoe@email.com', None, np.nan, 'Anonymous@email.com', 'NA'],
    'age': ['33', '55', '63', '36', None, None, 'Missing']
}

df = pd.DataFrame(people)

In [45]:
df

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
4,NaN,NaN,NaN,None
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


In [46]:
df.dropna()

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
6,NA,Missing,NA,Missing


In [48]:
df.dropna(axis='index',how='all')

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


In [50]:
df.dropna(axis='index',how='any',subset=['email']) # only drop missing value in the email column 

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


In [51]:
df.dropna(axis='index',how='all',subset=['last','email']) 
# keep rows with either non-mising email or non-mising last name
# namely, drop rows with the two fields missing

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
5,None,NaN,Anonymous@email.com,None
6,NA,Missing,NA,Missing


#### replace customized NA values with Numpy NaN

In [53]:
df.replace('NA', np.nan, inplace=True)
df.replace('Missing', np.nan, inplace=True)

In [54]:
df.dropna(axis='index',how='all')

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,None,36
5,None,NaN,Anonymous@email.com,None


#### replace customized NA values when importing data from csv files.

In [66]:
na_vals = ['NA', 'Missing']
df = pd.read_csv('survey_results_public.csv', index_col='Respondent', na_values=na_vals)
schema_df = pd.read_csv('survey_results_schema.csv', index_col='Column')

#### find NA

In [56]:
df.isna()

,first,last,email,age
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,True,False
4,True,True,True,True
5,True,True,False,True
6,True,True,True,True


#### replace all missing value with a value

In [57]:
df.fillna('MISSING')

,first,last,email,age
0,Corey,Schafer,CoreyMSchafer@gmail.com,33
1,Jane,Doe,JaneDoe@email.com,55
2,John,Doe,JohnDoe@email.com,63
3,Chris,Schafer,MISSING,36
4,MISSING,MISSING,MISSING,MISSING
5,MISSING,MISSING,Anonymous@email.com,MISSING
6,MISSING,MISSING,MISSING,MISSING


# Casting Datatypes

In [58]:
df.dtypes

first    object
last     object
email    object
age      object
dtype: object

In [59]:
# if we have NaN values in a column that we are trying to convert to numbers
# you need to use the float datatype. it is because the date type of NaN is actually Float

In [62]:
type(np.nan)

float

In [60]:
df['age'].mean()

TypeError: can only concatenate str (not "int") to str

In [63]:
df['age'] = df['age'].astype(float)

In [64]:
df['age'].mean()

46.75

# Date and Time Series Data

In [104]:
import pandas as pd
df = pd.read_csv('ETH_1h.csv')
df

,Date,Symbol,Open,High,Low,Close,Volume
0,2020-03-13 08-PM,ETHUSD,129.94,131.82,126.87,128.71,1940673.93
1,2020-03-13 07-PM,ETHUSD,119.51,132.02,117.10,129.94,7579741.09
2,2020-03-13 06-PM,ETHUSD,124.47,124.85,115.50,119.51,4898735.81
3,2020-03-13 05-PM,ETHUSD,124.08,127.42,121.63,124.47,2753450.92
4,2020-03-13 04-PM,ETHUSD,124.85,129.51,120.17,124.08,4461424.71
...,...,...,...,...,...,...,...
23669,2017-07-01 03-PM,ETHUSD,265.74,272.74,265.00,272.57,1500282.55
23670,2017-07-01 02-PM,ETHUSD,268.79,269.90,265.00,265.74,1702536.85
23671,2017-07-01 01-PM,ETHUSD,274.83,274.93,265.00,268.79,3010787.99
23672,2017-07-01 12-PM,ETHUSD,275.01,275.01,271.00,274.83,824362.87


In [71]:
df['Date'] = pd.to_datatime(df['Date'])
# pandas would do its best to figure out the formatting of the date/time and converted accordingly
# but the original data is in a pretty different format so it is not gonna work

AttributeError: module 'pandas' has no attribute 'to_datatime'

#### Convert string to datetime

In [105]:
df['Date'] = pd.to_datetime(df['Date'],format = '%Y-%m-%d %I-%p')
# code:https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [79]:
df

,Date,Symbol,Open,High,Low,Close,Volume
0,2020-03-13 08-PM,ETHUSD,129.94,131.82,126.87,128.71,1940673.93
1,2020-03-13 07-PM,ETHUSD,119.51,132.02,117.10,129.94,7579741.09
2,2020-03-13 06-PM,ETHUSD,124.47,124.85,115.50,119.51,4898735.81
3,2020-03-13 05-PM,ETHUSD,124.08,127.42,121.63,124.47,2753450.92
4,2020-03-13 04-PM,ETHUSD,124.85,129.51,120.17,124.08,4461424.71
...,...,...,...,...,...,...,...
23669,2017-07-01 03-PM,ETHUSD,265.74,272.74,265.00,272.57,1500282.55
23670,2017-07-01 02-PM,ETHUSD,268.79,269.90,265.00,265.74,1702536.85
23671,2017-07-01 01-PM,ETHUSD,274.83,274.93,265.00,268.79,3010787.99
23672,2017-07-01 12-PM,ETHUSD,275.01,275.01,271.00,274.83,824362.87


In [76]:
df.loc[0,'Date'].day_name()

'Friday'

#### Convert string to datetime when the data is read in

In [80]:
d_parser = lambda x:pd.datetime.strptime(x,'%Y-%m-%d %I-%p')
df = pd.read_csv('ETH_1h.csv',parse_dates = ['Date'], date_parser = d_parser)

In [81]:
df.head()

,Date,Symbol,Open,High,Low,Close,Volume
0,2020-03-13 20:00:00,ETHUSD,129.94,131.82,126.87,128.71,1940673.93
1,2020-03-13 19:00:00,ETHUSD,119.51,132.02,117.10,129.94,7579741.09
2,2020-03-13 18:00:00,ETHUSD,124.47,124.85,115.50,119.51,4898735.81
3,2020-03-13 17:00:00,ETHUSD,124.08,127.42,121.63,124.47,2753450.92
4,2020-03-13 16:00:00,ETHUSD,124.85,129.51,120.17,124.08,4461424.71


#### use DT class on the series to get day name for a series

In [82]:
df['Date'].dt.day_name()

0          Friday
1          Friday
2          Friday
3          Friday
4          Friday
           ...   
23669    Saturday
23670    Saturday
23671    Saturday
23672    Saturday
23673    Saturday
Name: Date, Length: 23674, dtype: object

In [83]:
df.dtypes

Date      datetime64[ns]
Symbol            object
Open             float64
High             float64
Low              float64
Close            float64
Volume           float64
dtype: object

In [84]:
# 1. we can use string here, pandas will understand we mean the year of 2019
filt = (df['Date'] >= '2019') & (df['Date'] < '2020')  
df.loc[filt]

,Date,Symbol,Open,High,Low,Close,Volume
1749,2019-12-31 23:00:00,ETHUSD,128.33,128.69,128.14,128.54,440678.91
1750,2019-12-31 22:00:00,ETHUSD,128.38,128.69,127.95,128.33,554646.02
1751,2019-12-31 21:00:00,ETHUSD,127.86,128.43,127.72,128.38,350155.69
1752,2019-12-31 20:00:00,ETHUSD,127.84,128.34,127.71,127.86,428183.38
1753,2019-12-31 19:00:00,ETHUSD,128.69,128.69,127.60,127.84,1169847.84
...,...,...,...,...,...,...,...
10504,2019-01-01 04:00:00,ETHUSD,130.75,133.96,130.74,131.96,2791135.37
10505,2019-01-01 03:00:00,ETHUSD,130.06,130.79,130.06,130.75,503732.63
10506,2019-01-01 02:00:00,ETHUSD,130.79,130.88,129.55,130.06,838183.43
10507,2019-01-01 01:00:00,ETHUSD,131.62,131.62,130.77,130.79,434917.99


In [87]:
# 2. we can use to_datetime too
filt = (df['Date'] >= pd.to_datetime('2019-01-01')) & (df['Date'] < pd.to_datetime('2020-01-01'))
df.loc[filt]

,Date,Symbol,Open,High,Low,Close,Volume
1749,2019-12-31 23:00:00,ETHUSD,128.33,128.69,128.14,128.54,440678.91
1750,2019-12-31 22:00:00,ETHUSD,128.38,128.69,127.95,128.33,554646.02
1751,2019-12-31 21:00:00,ETHUSD,127.86,128.43,127.72,128.38,350155.69
1752,2019-12-31 20:00:00,ETHUSD,127.84,128.34,127.71,127.86,428183.38
1753,2019-12-31 19:00:00,ETHUSD,128.69,128.69,127.60,127.84,1169847.84
...,...,...,...,...,...,...,...
10504,2019-01-01 04:00:00,ETHUSD,130.75,133.96,130.74,131.96,2791135.37
10505,2019-01-01 03:00:00,ETHUSD,130.06,130.79,130.06,130.75,503732.63
10506,2019-01-01 02:00:00,ETHUSD,130.79,130.88,129.55,130.06,838183.43
10507,2019-01-01 01:00:00,ETHUSD,131.62,131.62,130.77,130.79,434917.99


#### slice data by setting date as index

In [93]:
df.set_index('Date',inplace=True)

In [95]:
df['2020-01':'2020-02']

,Symbol,Open,High,Low,Close,Volume
Date,,,,,,
2020-02-29 23:00:00,ETHUSD,223.35,223.58,216.83,217.31,1927939.88
2020-02-29 22:00:00,ETHUSD,223.48,223.59,222.14,223.35,535998.57
2020-02-29 21:00:00,ETHUSD,224.63,225.14,222.74,223.48,561158.03
2020-02-29 20:00:00,ETHUSD,225.31,225.33,223.50,224.63,511648.65
2020-02-29 19:00:00,ETHUSD,225.09,225.85,223.87,225.31,1250856.20
...,...,...,...,...,...,...
2020-01-01 04:00:00,ETHUSD,129.57,130.00,129.50,129.56,702786.82
2020-01-01 03:00:00,ETHUSD,130.37,130.44,129.38,129.57,496704.23
2020-01-01 02:00:00,ETHUSD,130.14,130.50,129.91,130.37,396315.72


### resample

In [106]:
# resample can group datetimes to different levels
# Object must have a datetime-like index
# codes : https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

In [96]:
df['2020-01-01']['High'].max()

132.68

In [101]:
df['High'].resample('D').max()['2020-01-01']
# the datetime was at hour level, the method groups the datetimes to the day level 

132.68

#### apply different aggregation methods to different columns

In [102]:
df.resample('W').agg({'Close':'mean','High':'max'})

,Close,High
Date,,
2017-07-02,268.202162,293.73
2017-07-09,261.062083,285.00
2017-07-16,195.698393,240.33
2017-07-23,212.783750,249.40
2017-07-30,203.309524,229.99
...,...,...
2020-02-16,255.198452,290.00
2020-02-23,265.321905,287.13
2020-03-01,236.373988,278.13
